In [1]:
import os
import sys

# Set the root directory of your project
project_root = '/Users/rodolfocacacho/Documents/Documents/MAI/Master Thesis/Code/rag_project'
os.chdir(project_root)
# Add the root directory to sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

from config import (CONFIG_SQL_DB,DB_NAME,
                    SQL_EVAL_QAS_TABLE_SCHEMA,
                    SQL_EVAL_QAS_TABLE, 
                    EMBEDDING_MODEL,EMBEDDING_MODEL_API,
                    EMBEDDING_MODEL_EMB_TASK,
                    TEST_RESULTS_TABLE,SQL_EVAL_CHUNKS_TABLE,
                    SQL_PROMPTS_TABLE,TEST_GEN_ANSWERS_TABLE)
from utils.MySQLDB_manager import MySQLDB
from testing.modules.evaluating_modules import RAGEvaluator
import json

sql_con = MySQLDB(CONFIG_SQL_DB,DB_NAME)

ragEval = RAGEvaluator(sql_con=sql_con,
                       test_table_name=TEST_RESULTS_TABLE,
                       qas_table_name=SQL_EVAL_QAS_TABLE,
                       chunks_eval_table_name=SQL_EVAL_CHUNKS_TABLE,
                       prompts_table_name=SQL_PROMPTS_TABLE,
                       eval_answers_table=TEST_GEN_ANSWERS_TABLE)

df_results = ragEval.data_df

print(df_results.head(5))
print(df_results.columns)
print(df_results.shape)

# print(ragEval.generate_report())


   id_question  id_sample type_question  \
0            1          1       Factual   
1            1          1       Factual   
2            1          1       Factual   
3            1          1       Factual   
4            1          1       Factual   

                                            question  \
0  Welche Dokumente müssen bei der Beantragung de...   
1  Welche Dokumente müssen bei der Beantragung de...   
2  Welche Dokumente müssen bei der Beantragung de...   
3  Welche Dokumente müssen bei der Beantragung de...   
4  Welche Dokumente müssen bei der Beantragung de...   

                                     expected_answer  clarity  specificity  \
0  Bei der Beantragung des Klimageschwindigkeits-...        5            5   
1  Bei der Beantragung des Klimageschwindigkeits-...        5            5   
2  Bei der Beantragung des Klimageschwindigkeits-...        5            5   
3  Bei der Beantragung des Klimageschwindigkeits-...        5            5   
4  Bei der Bea

In [7]:
import pandas as pd

def generate_test_report(data_df, groupby_cols=None, output_file="test_report.xlsx"):
    """
    Generate detailed test reports with multiple groups and export them to an Excel file.
    
    Args:
        data_df (pd.DataFrame): The dataframe containing the test data.
        groupby_cols (list of str): Columns to create individual groupings for each test.
        output_file (str): Path to the output Excel file.
        
    Returns:
        None
    """
    if groupby_cols is None:
        groupby_cols = []

    # Start writing to Excel
    with pd.ExcelWriter(output_file) as writer:
        # Group by 'test_name'
        for test_name, test_group in data_df.groupby("test_name"):
            # General stats for the test (without grouping)
            total_Qs = len(test_group)
            avg_prompt_tokens = test_group['prompt_tokens'].mean()
            std_prompt_tokens = test_group['prompt_tokens'].std()
            avg_answer_tokens = test_group['completion_tokens'].mean()
            std_answer_tokens = test_group['completion_tokens'].std()
            avg_proc_time = (test_group['end_date'] - test_group['begin_date']).mean().total_seconds()
            std_proc_time = (test_group['end_date'] - test_group['begin_date']).std().total_seconds()

            general_stats = pd.DataFrame([{
                'Total Questions': total_Qs,
                'Avg Prompt Tokens': avg_prompt_tokens,
                'Std Prompt Tokens': std_prompt_tokens,
                'Avg Answer Tokens': avg_answer_tokens,
                'Std Answer Tokens': std_answer_tokens,
                'Avg Processing Time (s)': avg_proc_time,
                'Std Processing Time (s)': std_proc_time,
            }])
            general_stats.to_excel(writer, sheet_name=f"{test_name}_General", index=False)

            # Individual groupings for each column in `groupby_cols`
            for col in groupby_cols:
                group_stats = []
                for group, group_data in test_group.groupby(col):
                    total_Qs = len(group_data)
                    avg_prompt_tokens = group_data['prompt_tokens'].mean()
                    std_prompt_tokens = group_data['prompt_tokens'].std()
                    avg_answer_tokens = group_data['completion_tokens'].mean()
                    std_answer_tokens = group_data['completion_tokens'].std()
                    avg_proc_time = (group_data['end_date'] - group_data['begin_date']).mean().total_seconds()
                    std_proc_time = (group_data['end_date'] - group_data['begin_date']).std().total_seconds()

                    group_stats.append({
                        col: group,
                        'Total Questions': total_Qs,
                        'Avg Prompt Tokens': avg_prompt_tokens,
                        'Std Prompt Tokens': std_prompt_tokens,
                        'Avg Answer Tokens': avg_answer_tokens,
                        'Std Answer Tokens': std_answer_tokens,
                        'Avg Processing Time (s)': avg_proc_time,
                        'Std Processing Time (s)': std_proc_time,
                    })

                # Convert group stats to DataFrame and write to a sheet
                group_stats_df = pd.DataFrame(group_stats)
                sheet_name = f"{test_name}_{col[:25]}"  # Ensure sheet name fits Excel's 31-char limit
                group_stats_df.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"Report saved to {output_file}")

In [ ]:
print(df_results.columns)


Index(['id_question', 'id_sample', 'type_question', 'question',
       'expected_answer', 'clarity', 'specificity', 'relevance', 'clarity_q',
       'specificity_q', 'relevance_q', 'prompt_id', 'device', 'test_name',
       'doc_type', 'source', 'content', 'id', 'metadata', 'answer',
       'begin_date', 'end_date', 'completion_tokens', 'prompt_tokens',
       'context_used', 'context_ids', 'context_ids_total', 'alternate_prompts',
       'improved_query', 'query_intent', 'keyterms'],
      dtype='object')


In [8]:
generate_test_report(df_results,groupby_cols=['type_question','doc_type','query_intent'])

Report saved to test_report.xlsx


In [4]:
import pandas as pd
import json

def calculate_custom_metrics(data_df):
    """
    Preprocess the dataframe by adding custom metric columns.
    """
    def get_adj_ids(id, last_id, dif=1):
        adjs_ids = []
        base, cid = id.split('.')
        _, lid = last_id.split('.')
        cid = int(cid)
        lid = int(lid)
        for offset in range(-dif, dif + 1):
            adj_cid = cid + offset
            if 0 <= adj_cid <= lid and adj_cid != cid:
                adjs_ids.append(f"{base}.{adj_cid}")
        return adjs_ids

    def row_metrics(row):
        metadata = json.loads(row['metadata'])
        end_chunk = metadata['last_id']
        context_ids = json.loads(row['context_ids'])
        total_context_ids = json.loads(row['context_ids_total'])
        adj_context = get_adj_ids(row['id'], end_chunk, dif=1)
        adj_context.append(row['id'])
        # Calculate binary score
        binary_score = 1 if row['score'] > 3 else 0

        return {
            'used_context': row['id'] in context_ids,
            'retrieved_context': row['id'] in total_context_ids,
            'used_context_ext': any(item in adj_context for item in context_ids),
            'retrieved_context_ext': any(item in adj_context for item in total_context_ids),
            'total_context_ids': len(total_context_ids),
            'binary_score': binary_score
        }

    # Apply the row-wise custom metric calculations
    metrics = data_df.apply(row_metrics, axis=1)
    metrics_df = pd.DataFrame(metrics.tolist())  # Convert list of dicts to a DataFrame
    return pd.concat([data_df.reset_index(drop=True), metrics_df], axis=1)

def generate_compact_test_report(data_df, groupby_cols=None, output_file="test_report.xlsx"):
    """
    Generate test reports with dynamic metric calculation and export to Excel.
    
    Args:
        data_df (pd.DataFrame): The dataframe containing test data with precomputed metrics.
        groupby_cols (list of str): Columns to group by within each test.
        output_file (str): Path to the output Excel file.
        
    Returns:
        None
    """
    if groupby_cols is None:
        groupby_cols = []

    with pd.ExcelWriter(output_file) as writer:
        for test_name, test_group in data_df.groupby("test_name"):
            # General Stats
            general_stats = pd.DataFrame([{
                'Total Questions': len(test_group),
                'Avg Prompt Tokens': test_group['prompt_tokens'].mean(),
                'Std Prompt Tokens': test_group['prompt_tokens'].std(),
                'Avg Answer Tokens': test_group['completion_tokens'].mean(),
                'Std Answer Tokens': test_group['completion_tokens'].std(),
                'Retrieval Accuracy (used)': test_group['used_context'].mean(),
                'Retrieval Accuracy (retrieved)': test_group['retrieved_context'].mean(),
                'Expanded Retrieval Accuracy (used)': test_group['used_context_ext'].mean(),
                'Expanded Retrieval Accuracy (retrieved)': test_group['retrieved_context_ext'].mean()
            }])
            general_stats.to_excel(writer, sheet_name=f"{test_name}_General", index=False)

            # Individual Groupings
            for col in groupby_cols:
                group_stats = []
                for group, group_data in test_group.groupby(col):
                    group_stats.append({
                        col: group,
                        'Total Questions': len(group_data),
                        'Avg Prompt Tokens': group_data['prompt_tokens'].mean(),
                        'Std Prompt Tokens': group_data['prompt_tokens'].std(),
                        'Avg Answer Tokens': group_data['completion_tokens'].mean(),
                        'Std Answer Tokens': group_data['completion_tokens'].std(),
                        'Retrieval Accuracy (used)': group_data['used_context'].mean(),
                        'Retrieval Accuracy (retrieved)': group_data['retrieved_context'].mean(),
                        'Expanded Retrieval Accuracy (used)': group_data['used_context_ext'].mean(),
                        'Expanded Retrieval Accuracy (retrieved)': group_data['retrieved_context_ext'].mean()
                    })

                group_stats_df = pd.DataFrame(group_stats)
                sheet_name = f"{test_name}_{col[:25]}"  # Ensure Excel sheet name limit
                group_stats_df.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"Report saved to {output_file}")

In [ ]:
def generate_consolidated_test_report(data_df, groupby_cols=None, output_file="consolidated_test_report.xlsx"):
    """
    Generate a consolidated test report with all tests in a single sheet for each group.

    Args:
        data_df (pd.DataFrame): The dataframe containing test data with precomputed metrics.
        groupby_cols (list of str): Columns to group by for metrics.
        output_file (str): Path to the output Excel file.

    Returns:
        None
    """
    if groupby_cols is None:
        groupby_cols = []

    with pd.ExcelWriter(output_file) as writer:
        # General Summary Sheet
        summary_stats = []
        for test_name, test_group in data_df.groupby("test_name"):
            summary_stats.append({
                'Test Name': test_name,
                'Total Questions': len(test_group),
                'Avg Prompt Tokens': test_group['prompt_tokens'].mean(),
                'Std Prompt Tokens': test_group['prompt_tokens'].std(),
                'Avg Answer Tokens': test_group['completion_tokens'].mean(),
                'Std Answer Tokens': test_group['completion_tokens'].std(),
                'Retrieval Accuracy (used)': test_group['used_context'].mean(),
                'Retrieval Accuracy (retrieved)': test_group['retrieved_context'].mean(),
                'Expanded Retrieval Accuracy (used)': test_group['used_context_ext'].mean(),
                'Expanded Retrieval Accuracy (retrieved)': test_group['retrieved_context_ext'].mean(),
                'Avg Score': test_group['score'].mean(),
                'Binary Score': test_group['binary_score'].mean()
            })
        summary_df = pd.DataFrame(summary_stats)
        summary_df.to_excel(writer, sheet_name="Summary", index=False)

        # Group Sheets
        for col in groupby_cols:
            group_stats = []
            for test_name, test_group in data_df.groupby("test_name"):
                for group, group_data in test_group.groupby(col):
                    group_stats.append({
                        'Test Name': test_name,
                        col: group,
                        'Total Questions': len(group_data),
                        'Avg Prompt Tokens': group_data['prompt_tokens'].mean(),
                        'Std Prompt Tokens': group_data['prompt_tokens'].std(),
                        'Avg Answer Tokens': group_data['completion_tokens'].mean(),
                        'Std Answer Tokens': group_data['completion_tokens'].std(),
                        'Retrieval Accuracy (used)': group_data['used_context'].mean(),
                        'Retrieval Accuracy (retrieved)': group_data['retrieved_context'].mean(),
                        'Expanded Retrieval Accuracy (used)': group_data['used_context_ext'].mean(),
                        'Expanded Retrieval Accuracy (retrieved)': group_data['retrieved_context_ext'].mean(),
                        'Avg Score': test_group['score'].mean(),
                        'Binary Score': test_group['binary_score'].mean()
                    })

            # Convert to DataFrame and write to a single sheet per group
            group_stats_df = pd.DataFrame(group_stats)
            sheet_name = col[:31]  # Ensure sheet name fits Excel's character limit
            group_stats_df.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"Consolidated report saved to {output_file}")

In [13]:
# Preprocess data to calculate metrics
processed_df = calculate_custom_metrics(df_results)

generate_compact_test_report(processed_df,groupby_cols=['type_question','doc_type','query_intent'],)



Report saved to test_report.xlsx


In [15]:
generate_consolidated_test_report(processed_df,groupby_cols=['type_question','doc_type','query_intent'])

Consolidated report saved to consolidated_test_report.xlsx


In [3]:
df_results

,id_question,id_sample,type_question,question,expected_answer,clarity,specificity,relevance,clarity_q,specificity_q,...,improved_query,query_intent,keyterms,score,comment,used_context,retrieved_context,used_context_ext,retrieved_context_ext,total_context_ids
0,1,1,Factual,Welche Dokumente müssen bei der Beantragung de...,Bei der Beantragung des Klimageschwindigkeits-...,5,5,5,4,4,...,"Welche Unterlagen sind erforderlich, um den Kl...",requirements,"[""Dokumente"", ""Beantragung"", ""Klimageschwindig...",5,The generated response is fully accurate and a...,False,False,False,False,34
1,1,1,Factual,Welche Dokumente müssen bei der Beantragung de...,Bei der Beantragung des Klimageschwindigkeits-...,5,5,5,4,4,...,Welche Unterlagen sind für den Antrag des Klim...,requirements,"[""Dokumente"", ""Unterlagen"", ""Antragsunterlagen...",4,The generated response aligns well with the Ex...,False,False,False,False,37
2,1,1,Factual,Welche Dokumente müssen bei der Beantragung de...,Bei der Beantragung des Klimageschwindigkeits-...,5,5,5,4,4,...,Welche Unterlagen sind für die Beantragung des...,requirements,"[""Unterlagen"", ""Klimageschwindigkeits-Bonus"", ...",4,The generated response accurately lists the tw...,False,False,False,False,84
3,1,1,Factual,Welche Dokumente müssen bei der Beantragung de...,Bei der Beantragung des Klimageschwindigkeits-...,5,5,5,4,4,...,Welche Unterlagen sind für die Beantragung des...,requirements,"[""Klimageschwindigkeits-Bonus"", ""Beantragung"",...",5,The response accurately lists the required doc...,False,True,False,True,33
4,1,1,Factual,Welche Dokumente müssen bei der Beantragung de...,Bei der Beantragung des Klimageschwindigkeits-...,5,5,5,4,4,...,Welche Unterlagen sind für die Beantragung des...,requirements,"[""Dokumente"", ""Beantragung"", ""Unterlagen"", ""Kl...",5,The generated response is fully accurate and a...,False,True,False,True,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2763,635,127,Analytical,"Warum ist es notwendig, eine tabellarische Bel...",1. **Notwendigkeit der tabellarischen Belegübe...,4,5,5,5,5,...,Weshalb muss bei der Einreichung des Verwendun...,requirements,"[""Ausgabenübersicht"", ""Verantwortungsnachweis""...",4,The generated response accurately covers the n...,False,False,False,True,57
2764,635,127,Analytical,"Warum ist es notwendig, eine tabellarische Bel...",1. **Notwendigkeit der tabellarischen Belegübe...,4,5,5,5,5,...,Welche Informationen muss eine tabellarische B...,compliance,"[""Belegübersicht"", ""Verwendungsnachweis"", ""Bun...",5,The generated response accurately captures the...,True,True,True,True,56
2765,635,127,Analytical,"Warum ist es notwendig, eine tabellarische Bel...",1. **Notwendigkeit der tabellarischen Belegübe...,4,5,5,5,5,...,Welche Informationen muss die tabellarische Be...,requirements,"[""tabellarische Belegübersicht"", ""Einreichung""...",4,The generated response is largely accurate and...,True,True,True,True,63
2766,635,127,Analytical,"Warum ist es notwendig, eine tabellarische Bel...",1. **Notwendigkeit der tabellarischen Belegübe...,4,5,5,5,5,...,Warum muss eine tabellarische Belegübersicht b...,compliance,"[""Verwendungsnachweis"", ""Tabelle der Belege"", ...",5,The generated response is fully accurate and a...,True,True,True,True,39
